<a href="https://colab.research.google.com/github/Exion007/Colab/blob/main/sample_training3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Training

In [283]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [284]:
df = pd.read_csv("movies.csv")
df.head(3)

,id,title,release_date,genres,original_language,vote_average,vote_count,popularity,overview,budget,production_companies,revenue,runtime,tagline
0,758323,The Pope's Exorcist,2023-04-05,"['Horror', 'Mystery', 'Thriller']",English,7.4,619,5089.969,"Father Gabriele Amorth, Chief Exorcist of the ...",18000000,"['Screen Gems', '2.0 Entertainment', 'Jesus & ...",65675816,103,Inspired by the actual files of Father Gabriel...
1,640146,Ant-Man and the Wasp: Quantumania,2023-02-15,"['Action', 'Adventure', 'Science Fiction']",English,6.6,2294,4665.438,Super-Hero partners Scott Lang and Hope van Dy...,200000000,"['Marvel Studios', 'Kevin Feige Productions']",464566092,125,Witness the beginning of a new dynasty.
2,502356,The Super Mario Bros. Movie,2023-04-05,"['Animation', 'Adventure', 'Family', 'Fantasy'...",English,7.5,1861,3935.550,"While working underground to fix a water main,...",100000000,"['Universal Pictures', 'Illumination', 'Ninten...",1121048165,92,NaN


In [285]:
df = df.loc[:, ['vote_average', 'vote_count', 'genres', 'budget', 'popularity']]
df.dropna(inplace = True)
df = df.query('budget > 0')
print(df.shape[0], "\n")
df.head(3)

5136 



,vote_average,vote_count,genres,budget,popularity
0,7.4,619,"['Horror', 'Mystery', 'Thriller']",18000000,5089.969
1,6.6,2294,"['Action', 'Adventure', 'Science Fiction']",200000000,4665.438
2,7.5,1861,"['Animation', 'Adventure', 'Family', 'Fantasy'...",100000000,3935.550


In [286]:
def to_list(row):
  row = row[1:-1].replace("'", "").split(", ")
  return row

df['genres'] = df['genres'].apply(to_list)

df.head(3)

,vote_average,vote_count,genres,budget,popularity
0,7.4,619,"[Horror, Mystery, Thriller]",18000000,5089.969
1,6.6,2294,"[Action, Adventure, Science Fiction]",200000000,4665.438
2,7.5,1861,"[Animation, Adventure, Family, Fantasy, Comedy]",100000000,3935.550


In [287]:
X = df[['vote_count', 'genres', 'budget', 'popularity']]
y = df['vote_average']

mlb = MultiLabelBinarizer()

# Transform the genres column into binary columns
genres_encoded = pd.DataFrame(mlb.fit_transform(X['genres']), columns=mlb.classes_, index=X.index)

# Concatenate the encoded genres with the original features
X_encoded = pd.concat([X[['vote_count', 'budget', 'popularity']], genres_encoded], axis=1)


In [288]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [ ]:
model = Pipeline([
    ('regressor', RandomForestRegressor())
])
model.fit(X_train, y_train)

In [290]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.41653380155642017


In [294]:
def vote_inputs():
  genre_ids = []
  x = True
  while (x):
    genre_id = int(input("Enter genre id: "))

    if (genre_id == -1):
      x = False

    else:
      if genre_id not in genre_ids:
        genre_ids.append(genre_id)
  return genre_ids

#new_vote_count = int(input("Enter vote_count: "))
#new_popularity = int(input("Enter popularity: "))
#new_genre_ids = vote_inputs()


# Example prediction for new data
new_data = pd.DataFrame({
    'vote_count': [9000],
    'genres': [['Action', 'Adventure']],
    'budget': [100000],
    'popularity': [23450.0]
})

# Transform the genres for the new data
new_data_genres_encoded = pd.DataFrame(mlb.transform(new_data['genres']), columns=mlb.classes_, index=new_data.index)

# Concatenate the encoded genres with the new data features
new_data_encoded = pd.concat([new_data[['vote_count', 'budget', 'popularity']], new_data_genres_encoded], axis=1)

# Make prediction for new data
prediction = model.predict(new_data_encoded)
print(f"Predicted Vote Average: {prediction}")

Predicted Vote Average: [7.755]
